In [87]:
!pip install --upgrade openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [88]:
!pip install anthropic

In [89]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [90]:
from google.cloud import secretmanager
import os

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "latest")

In [91]:
import openai
openai.api_key = openai_key

In [92]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [93]:
import gcsfs
# load the cleaned dataset
fs = gcsfs.GCSFileSystem(project="sql_autograding")
# with fs.open("gs://sql_autograding/clean_submissions_final.csv") as f:
#     data = pd.read_csv(f)

In [94]:
submission_files = fs.ls("gs://thoth_v001/thoth_submissions")
submission_files = [f for f in submission_files if f.endswith(".csv")]
pattern = re.compile(r'thoth_submissions-q30\d{2}\.csv')
submission_files = [file for file in submission_files if not pattern.search(file)]

data_dfs = []

for file in submission_files:
    with fs.open(file) as f:
        df = pd.read_csv(f)
        data_dfs.append(df)

data = pd.concat(data_dfs, ignore_index=True)

In [95]:
data = data.drop(columns="Unnamed: 0")
data.head()

,QuestionID,SubmittedAnswer,StudentID,Score
0,0,select artist_name from artist order by arti...,bmt4,1.0
1,0,select artist_name from artist order by arti...,bmt4,1.0
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0
3,0,select artist_name as artist from artist ord...,md4968,0.5
4,0,select artist_name from artist order by artist...,my2204,1.0


In [96]:
data.shape

(13295, 4)

In [97]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [98]:
#with fs.open("gs://sql_autograding/Database Homeworks.csv") as f:
with fs.open("gs://thoth_v001/database_homeworks.csv") as f:
    hw = pd.read_csv(f)

hw.head()

,QuestionID,HomeWork,Question,Database,AnswerKey
0,0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
1,1,Assignment 2B: Selection Queries,List the ten shortest tracks in terms of playi...,music,SELECT * FROM track ORDER BY time LIMIT 10
2,2,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR..."
3,3,Assignment 2B: Selection Queries,Show all the albums.,music,SELECT * FROM album
4,4,Assignment 2B: Selection Queries,Show all the artists.,music,SELECT * FROM artist


In [99]:
df_all = pd.merge(data, hw, how='left', on='QuestionID')
print(df_all.loc[df_all['Database'].isna()]['QuestionID'].unique())
df_all = df_all.dropna()
df_all.head()

[119  31  53]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...


In [100]:
df_all['QuestionID'].unique()

array([  0,   1,  10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
        11, 110, 111, 112, 113, 114, 115, 116, 117, 118,  12,  13,  14,
        15,  16,  17,  18,  19,   2,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,   3,  30,  37,  38,  39,   4,  42,  43,  44,  45,
        46,  47,  48,  49,   5,  50,  51,  52,  54,  55,  56,  57,  58,
        59,   6,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,   7,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,   8,  80,  81,
        82,  83,  84,  85,  86,  87,  88,  89,   9,  90,  91,  92,  93,
        94,  95,  96,  97,  98,  99])

## Further preprocessing

In [101]:
db_list = df_all['Database'].unique().tolist()
print(db_list)

['music', 'flights', 'restaurants', 'imdb', 'northwind', 'collisions', 'facebook']


In [102]:
def db_schema(db_name):
    """
    Connects to a database and returns the schema of each table in the database.

    The function connects to a specific database using SQLAlchemy. It then retrieves the list of tables in
    the database and for each table, it queries the schema (i.e., the list of fields/columns) and stores it
    in a dictionary. The function returns a list of such dictionaries, with each dictionary representing a table
    and its corresponding schema.

    Args:
        db_name (str): The name of the database to connect to and retrieve schemas from.

    Returns:
        list: A list of dictionaries, with each dictionary containing the name of a table as the key and a
              list of its fields as the value.

    Example:
        >>> db_schema('flights')
        {'m_airports': ['airport', 'state', 'state_name']},
        {'m_ticket_prices': ['origin', 'dest', 'carrier', 'fare', 'fare_per_mile','passengers', 'distance']
        ...
        },
    """

    student_password = access_secret_version(
        "sql-autograding", "db_student_password", "1"
    )

    conn_string = (
        "mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4".format(
            host="db.ipeirotis.org",
            user="student",
            password=student_password,
            db=db_name,
            encoding="utf8mb4",
        )
    )
    engine = create_engine(conn_string)

    with engine.begin() as conn:
        tables = pd.read_sql_query(sql=text("show tables"), con=conn)
        tables = tables.iloc[:, -1].tolist()

        schema_list = []
        for t in tables:
            d = pd.read_sql_query(sql=text(f"describe {t}"), con=conn)
            table_schema = {t: d.loc[:, "Field"].tolist()}
            schema_list.append(table_schema)
    return schema_list

In [103]:
flights_schema = db_schema("flights")
imdb_schema = db_schema("imdb")
music_schema = db_schema("music")
restaurants_schema = db_schema("restaurants")
facebook_schema = db_schema("facebook")
northwind_schema = db_schema("northwind")
collisions_schema = db_schema("collisions")

In [104]:
# creates a dictionary that maps each schema to each database and add the schema as a new column in the dataframe
schema_mapping = {
    "flights": flights_schema,
    "imdb": imdb_schema,
    "music": music_schema,
    "restaurants": restaurants_schema,
    "facebook": facebook_schema,
    "northwind": northwind_schema,
    "collisions": collisions_schema,
}

df_all["schema"] = df_all["Database"].map(schema_mapping)

In [105]:
df_all['full_score'] = df_all.groupby('QuestionID')['Score'].transform('max')
df_all.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0


## Generating Mega Prompt
===============================

a. We are working with the XXX database, which has the following tables: artist(id, first_name, last_name) album(id, artist_id, name) track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back these responses and grades:

f1. submission: ..... , grade: ..... , feedback

f2. submission: ..... , grade: ..... , feedback

f3. submission: ..... , grade: ..... , feedback

Please provide a grade out of the full score of 10 and feedback for the student

===============================

In [106]:
from typing import List

def generate_submissions_string(
    train_df: pd.DataFrame, submission_size: int = None
) -> str:
    """
    Generate a string that concatenates student submissions up to the given size.

    Args:
        train_df (pd.DataFrame): The DataFrame containing the training data.
        submission_size (int, optional): The number of submissions to include in the string. If None, include all submissions.

    Returns:
        str: A string containing the concatenated submissions.
    """
    submissions = ""
    for i, row in train_df.iterrows():
        if submission_size is not None and i >= submission_size:
            break
        submission = row["SubmittedAnswer"]
        grade = row["Score"]
        full_grade = row["full_score"]
        submissions += f"e{i}. submission: {submission}, grade: {grade}/{full_grade}\\n\\n"
    return submissions

In [107]:
def generate_mega_prompt_for_id(
    df: pd.DataFrame, id: str, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        id (str): The QuestionId to generate the mega prompt for.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df[df["QuestionID"] == id]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]
      # full_score = temp.loc[0, "full_score"]
      test = temp.loc[0, "SubmittedAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade out of the full score of 10 and feedback for the student.

        """ + """Only provide a response following this format without deviation. {"grade": "the grade out of the 10 you give", "feedback": "the feedback you give"}"""
      return mega_p

    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = generate_submissions_string(train_df, submission_size)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    # full_score = temp.loc[0, "full_score"]
    test = test_df.loc[0, "SubmittedAnswer"]

    mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade out of the full score of 10 and feedback for the student.

        """ + """Only provide a response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
    return mega_p

In [108]:
def generate_mega_prompts(
    df: pd.DataFrame, submission_size: int = None
) -> pd.DataFrame:
    """
    Generate a DataFrame of mega prompts for each unique QuestionId in the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        submission_size (int, optional): The number of submissions to include in each mega prompt. If None, include all submissions.

    Returns:
        pd.DataFrame: A DataFrame containing the mega prompts.
    """
    result = []
    qids = df["QuestionID"].unique()

    for id in qids:
        mega_p = generate_mega_prompt_for_id(df, id, submission_size)
        new_row = {"QuestionID": id, "MegaPrompt": mega_p}
        result.append(new_row)

    mega_df = pd.DataFrame(result)

    return mega_df

In [109]:
from openai import OpenAI

# client = OpenAI(
#   api_key="<llama api token>",
#   base_url = "https://api.llama-api.com"
# )
client = OpenAI(
  api_key=openai_key,  # this is also the default, it can be omitted
)

In [189]:
from tenacity import(retry, stop_after_attempt, wait_random_exponential)
# Answering Mega Prompt Using GPT 4 model
# dealing with rate limit of GPT4
import time
# further control rate limit for GPT-4
rate_limit_per_minute = 40000
delay = 60.0 / rate_limit_per_minute

def delayed_completion(p):
    time.sleep(delay)
    return GPT4_generation_with_backoff(p)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4_generation_with_backoff(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].message.content

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4o_generation_with_backoff(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=10000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response.choices[0].message.content

In [112]:
# message = client.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=1024,
#     messages=[
#         {"role": "user", "content": "Hello, Claude"}
#     ]
# )

In [113]:
# print(message.content)

## DataFrame with Zero-Shot Prompts

In [114]:
df_all.shape

(12877, 10)

In [190]:
from tqdm import tqdm
import numpy as np
import json

# data1, _ = train_test_split(df_all, train_size=1600, stratify=df_all['QuestionID'], random_state=1234)

# use complete dataset
data1 = df_all.copy()
data1.reset_index(drop=True, inplace=True)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0


In [191]:
# A function that parses grade and feedback from GPT responses
def parse_grade_feedback(text):
  colon_pos = text.find(":")
  divider_pos = text.find("/")

  grade = text[colon_pos+2:divider_pos]

  feedback_start = text.find("Feedback: ")
  if feedback_start != -1:
      feedback = text[feedback_start + len("Feedback: "):]
  else:
      feedback = None
  return grade, feedback

In [192]:
# parse grade and feedback from json object
def extract_grade(js):
  try:
    js = json.loads(js)
    return js["grade"]
  except json.JSONDecodeError:
    return ''

def extract_feedback(js):
  try:
    js = json.loads(js)
    return js["feedback"]
  except json.JSONDecodeError:
    return ''

In [193]:
lst = []
for i,r in data1.iterrows():
  # generate zero-shot mega prompt for each row of the dataframe
  lst.append(generate_mega_prompts(r.to_frame().T,0).loc[0,'MegaPrompt'])
# append mega prompts as a new column to the dataframe
data1['0_mp'] = lst
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...


In [194]:
# tqdm.pandas()
# generate GPT4 responses for each row and parse grade and feedback as new columns
# data1['4_0_answer'] = data1['0_mp'].progress_apply(GPT4_generation_with_backoff)
# data1['4_0_grade'], data1['4_0_feedback'] = zip(*data1['4_0_answer'].apply(parse_grade_feedback))
# data1.head()

In [195]:
# Use parallelism to generate response
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to parallelize GPT4 response
def parallel_gpt4_generation(df, column_name, max_workers=10):
    # Prepare a list to hold the results in the correct order
    results = [None] * len(df)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the executor, mapping future to index
        futures = {executor.submit(GPT4o_generation_with_backoff, row[column_name]): idx for idx, row in df.iterrows()}

        # Collect the results as they complete, ensuring correct order
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]  # Get the original index of the row
            # print(f"Processing row index: {idx}")
            try:
                if idx < len(results):  # Ensure index is within the valid range
                    results[idx] = future.result()  # Assign the result to the correct position
            except Exception as e:
                if idx < len(results):
                    results[idx] = None  # Handle any errors and assign None to this position

    return results


# Generate GPT4 responses
data1['4o_answer_zero'] = parallel_gpt4_generation(data1, '0_mp', max_workers=15)
data1.head()

100%|██████████| 12877/12877 [19:49<00:00, 10.82it/s]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp,4o_answer_zero
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q..."
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q..."
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q..."
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""9"", ""feedback"": ""The student's quer..."
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q..."


In [196]:
# Extract grade and feedback as new columns
data1['4o_grade_zero'] = data1['4o_answer_zero'].apply(extract_grade)
data1['4o_feedback_zero'] = data1['4o_answer_zero'].apply(extract_feedback)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,0_mp,4o_answer_zero,4o_grade_zero,4o_feedback_zero
0,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q...",10,Great job! Your query correctly lists all the ...
1,0,select artist_name from artist order by arti...,bmt4,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q...",10,Great job! Your query correctly lists all the ...
2,0,SELECT artist_name FROM artist ORDER BY artist...,fam7,0.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q...",10,Great job! Your query correctly selects the ar...
3,0,select artist_name as artist from artist ord...,md4968,0.5,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""9"", ""feedback"": ""The student's quer...",9,The student's query is almost correct. The use...
4,0,select artist_name from artist order by artist...,my2204,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music dat...,"{""grade"": ""10"", ""feedback"": ""Great job! Your q...",10,Great job! Your query correctly selects the ar...


In [197]:
data1.to_csv('zero_shot_4.csv')

In [ ]:
# fs = gcsfs.GCSFileSystem(project="sql_autograding")
# with fs.open("gs://sql_autograding/zero_shot_4.csv") as f:
#     data1 = pd.read_csv(f)

## Generate rubric based on Feedbacks

In [202]:
def create_rubric_prompt(question_id, question, database, schema, answer_key, feedback_list):
    """
    Generate a prompt for GPT-4o to create a grading rubric based on feedback.

    Args:
        question_id (str): The unique identifier for the question.
        question (str): The question text.
        database (str): The database context.
        schema (str): Schema details for the question.
        answer_key (str): The model answer for the question.
        feedback_list (list): A list of feedback responses for the question.

    Returns:
        str: The generated prompt.
    """
    # Format feedbacks
    feedback_text = "\n".join([f"- Feedback for submission {i+1}: {feedback}" for i, feedback in enumerate(feedback_list) if feedback])

    # Construct the prompt with all the necessary details
    prompt = f"""
        We are working on grading student submissions for a specific question in a database context.

        Question ID: {question_id}
        Question: "{question}"
        Database working with: "{database}"
        Tables in the database: "{schema}"
        Model answer: "{answer_key}" (note that we may have multiple correct answers, the model answer is just an example)

        Each feedback below is given to a different submission and may correspond to either correct or incorrect answers. Based on these feedback entries, please help design a grading rubric that lists common issues and points to deduct out of a full point of 10.

        Feedback Entries:
        {feedback_text}

        Please only provide a grading rubric in your response strictly in JSON format following this format without deviation:

    """ + "{'issue': 'point to deduct'}" + "\n" + "Make sure the entire JSON is on a single line without any newline character"

    return prompt


In [203]:
# Function to process each QuestionID and generate a rubric
def generate_rubric_for_question(group):
    question_id = group['QuestionID'].iloc[0]
    feedback_list = group['4o_feedback_zero'].tolist()
    question = group['Question'].iloc[0]
    database = group['Database'].iloc[0]
    schema = group['schema'].iloc[0]
    answer_key = group['AnswerKey'].iloc[0]

    # Create the prompt with the necessary details
    prompt = create_rubric_prompt(question_id, question, database, schema, answer_key, feedback_list)

    # Generate the rubric using GPT-4o
    rubric_response = GPT4o_generation_with_backoff(prompt)

    return question_id, rubric_response

# Parallel processing of rubrics
def parallel_generate_rubrics(df, max_workers=10):
    rubrics = {}

    # Group by QuestionID
    groups = [group for _, group in df.groupby('QuestionID')]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to executor for each group
        futures = {executor.submit(generate_rubric_for_question, group): group['QuestionID'].iloc[0] for group in groups}

        # Collect the results as they complete, ensuring correct order
        for future in tqdm(as_completed(futures), total=len(futures)):
            question_id, rubric_response = future.result()
            rubrics[question_id] = rubric_response  # Store results in the dictionary

    # Convert to DataFrame
    rubric_df = pd.DataFrame(list(rubrics.items()), columns=['QuestionID', 'Rubric'])
    return rubric_df

# Run the parallel rubric generation
rubric_df = parallel_generate_rubrics(data1)
rubric_df.head()

100%|██████████| 110/110 [03:02<00:00,  1.66s/it]


,QuestionID,Rubric
0,5,"{'Missing additional queries': 5, 'Incorrect t..."
1,4,"{""Unnecessary database context commands (e.g.,..."
2,3,"{""Unnecessary commands (e.g., 'USE music;', 'S..."
3,0,```json\n{'Correct query with optional ASC': 0...
4,1,"```json\n{""Missing ORDER BY clause"": 3, ""Missi..."


In [220]:
def clean_rubric_format_simple(rubric):
    """
    Cleans the format of the rubric string by removing unwanted markdown markers and extra newlines.

    Args:
        rubric (str): The rubric string to clean.

    Returns:
        str: A cleaned single-line string.
    """
    # Remove ```json and ``` markers, if they exist
    cleaned_rubric = rubric.strip().replace("```json\n", "").replace("\n```", "")

    # Remove any remaining newline characters and extra spaces
    cleaned_rubric = cleaned_rubric.replace("\n", " ").strip()

    return cleaned_rubric

In [221]:
rubric_df['Rubric'].apply(clean_rubric_format_simple)

,Rubric
0,"{'Missing additional queries': 5, 'Incorrect t..."
1,"{""Unnecessary database context commands (e.g.,..."
2,"{""Unnecessary commands (e.g., 'USE music;', 'S..."
3,"{'Correct query with optional ASC': 0, 'Unnece..."
4,"{""Missing ORDER BY clause"": 3, ""Missing LIMIT ..."
...,...
105,"{""Correct use of LEFT JOIN"": 0, ""Incorrect use..."
106,"{""Correct query with no issues"": 0, ""Unnecessa..."
107,"{""Unnecessary use of JOIN or tables"": 1, ""Unne..."
108,"{""Missing ORDER BY clause"": 1, ""Incomplete joi..."


In [222]:
hw_w_rubric = pd.merge(hw, rubric_df, on='QuestionID', how='left')
hw_w_rubric.head()

,QuestionID,HomeWork,Question,Database,AnswerKey,Rubric
0,0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,```json\n{'Correct query with optional ASC': 0...
1,1,Assignment 2B: Selection Queries,List the ten shortest tracks in terms of playi...,music,SELECT * FROM track ORDER BY time LIMIT 10,"```json\n{""Missing ORDER BY clause"": 3, ""Missi..."
2,2,Assignment 2B: Selection Queries,Show all the album names and the corresponding...,music,"SELECT artist_id, album_name AS album_title FR...","{""Correct query with unnecessary commands (e.g..."
3,3,Assignment 2B: Selection Queries,Show all the albums.,music,SELECT * FROM album,"{""Unnecessary commands (e.g., 'USE music;', 'S..."
4,4,Assignment 2B: Selection Queries,Show all the artists.,music,SELECT * FROM artist,"{""Unnecessary database context commands (e.g.,..."


In [223]:
hw_w_rubric.to_csv('hw_w_rubric.csv', index=False)

## Few Shot


In [123]:
# A function that generates few-shot mega-prompts for a row given the expected number of submissions
from numpy import row_stack
def generate_fewshot_prompt_for_row(
    test, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        test (pd.DataFrame): The DataFrame - this row.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df_all[df_all["QuestionID"] == test['QuestionID']]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "SubmittedAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade out of the full score of 10 and feedback for the student

        """ + """Only provide a response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
      return mega_p

    # skip the current row
    temp = temp[temp['StudentID']!=test['StudentID']]
    if submission_size==None or submission_size >= temp.shape[0]:
      train_df = temp.copy().reset_index()
    else:
      # randomly sampling certain size of training rows for the same QuestionId
        train_df = temp.sample(n=submission_size).reset_index()

    submissions = generate_submissions_string(train_df)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    test = test['SubmittedAnswer']

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade out of the full score of 10 and feedback for the student.

      """ + """Only provide a response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
    return mega_p

In [124]:
data1, _ = train_test_split(df_all, train_size=800, stratify=df_all['QuestionID'], random_state=1234)
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score
2439,14,USE music; SELECT * FROM album WHERE artist...,dsa285,1.0,Assignment 2C: Filtering Queries,Using the id of Rolling Stones from Question C...,music,SELECT * FROM album WHERE artist_id = 4,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
125,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0
8427,52,"SELECT\tT.carrier, \t\tCOUNT(DISTINCT T.origi...",ac10819,10.0,Final Exam,Flights: Using the table flights.m_ticket_pric...,flights,"SELECT carrier, COUNT(*) AS routes, COUNT(DIST...","[{'airports': ['airport', 'state', 'state_name...",10.0
6200,30,"SELECT a.artist_id, a.artist_name, COUNT(a2.al...",ml8055,1.0,Assignment 4: Aggregations,Show the name of the artist and the number of ...,music,"SELECT R.artist_id, R.artist_name, COUNT(*) AS...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0
12567,9,select * from track where artist_id = 3 AND ...,li2116,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0


In [125]:
# Generate few-shot mega prompts with submission size = all for GPT4-32k model
lst = []
for i,r in data1.iterrows():
  lst.append(generate_fewshot_prompt_for_row(r))

data1['few_mp'] = lst
data1.head()

,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,few_mp
2439,14,USE music; SELECT * FROM album WHERE artist...,dsa285,1.0,Assignment 2C: Filtering Queries,Using the id of Rolling Stones from Question C...,music,SELECT * FROM album WHERE artist_id = 4,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...
125,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...
8427,52,"SELECT\tT.carrier, \t\tCOUNT(DISTINCT T.origi...",ac10819,10.0,Final Exam,Flights: Using the table flights.m_ticket_pric...,flights,"SELECT carrier, COUNT(*) AS routes, COUNT(DIST...","[{'airports': ['airport', 'state', 'state_name...",10.0,\n a. We are working with the flights dat...
6200,30,"SELECT a.artist_id, a.artist_name, COUNT(a2.al...",ml8055,1.0,Assignment 4: Aggregations,Show the name of the artist and the number of ...,music,"SELECT R.artist_id, R.artist_name, COUNT(*) AS...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...
12567,9,select * from track where artist_id = 3 AND ...,li2116,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...


In [126]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00


In [127]:
import tiktoken

# counting the number of tokens of mega-prompts
def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

In [128]:
# Counting the number of tokens for each mega prompt
data1['num'] = data1['few_mp'].apply(num_tokens_from_string)

In [130]:
# a function that takes the current row and maximum number of tokens we need
# need improvement since we select training rows randomly
def max_submission_size(t, max_size):
  idf = df_all[df_all["QuestionID"] == t['QuestionID']]
  for i in range(1,len(idf)-1):
    mp = generate_fewshot_prompt_for_row(t,i)
    if num_tokens_from_string(mp, "cl100k_base") >= max_size:
      return i-1
  return None

In [132]:
# 8196 tokens limit for gpt4, select a maximum submission size for each row
# 128k tokens limit for gpt4o, selection no longer needed
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  # s_size = max_submission_size(r, 100000)
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['few_mp'] = lst
data1.head()

100%|██████████| 400/400 [00:09<00:00, 43.30it/s]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,few_mp,num
2439,14,USE music; SELECT * FROM album WHERE artist...,dsa285,1.0,Assignment 2C: Filtering Queries,Using the id of Rolling Stones from Question C...,music,SELECT * FROM album WHERE artist_id = 4,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,7528
125,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,7477
8427,52,"SELECT\tT.carrier, \t\tCOUNT(DISTINCT T.origi...",ac10819,10.0,Final Exam,Flights: Using the table flights.m_ticket_pric...,flights,"SELECT carrier, COUNT(*) AS routes, COUNT(DIST...","[{'airports': ['airport', 'state', 'state_name...",10.0,\n a. We are working with the flights dat...,11276
6200,30,"SELECT a.artist_id, a.artist_name, COUNT(a2.al...",ml8055,1.0,Assignment 4: Aggregations,Show the name of the artist and the number of ...,music,"SELECT R.artist_id, R.artist_name, COUNT(*) AS...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,13294
12567,9,select * from track where artist_id = 3 AND ...,li2116,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,9833


In [133]:
# data1['num'] = data1['few_mp'].apply(num_tokens_from_string)
# data1.head()
print(data1['num'].max())

41157


In [136]:
# Use parallelism to generate few shot response

# Function to parallelize GPT4 response
def parallel_gpt4_generation(df, column_name, max_workers=10):
    # Prepare a list to hold the results in the correct order
    results = [None] * len(df)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the executor, mapping future to index
        futures = {executor.submit(GPT4o_generation_with_backoff, row[column_name]): idx for idx, row in df.iterrows()}

        # Collect the results as they complete, ensuring correct order
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]  # Get the original index of the row
            # print(f"Processing row index: {idx}")
            try:
                if idx < len(results):  # Ensure index is within the valid range
                    results[idx] = future.result()  # Assign the result to the correct position
            except Exception as e:
                print("error")
                if idx < len(results):
                    results[idx] = None  # Handle any errors and assign None to this position

    return results


# Generate GPT4 responses
data1['4o_answer_few'] = parallel_gpt4_generation(data1, 'few_mp', max_workers=15)
data1.head()

100%|██████████| 400/400 [00:50<00:00,  7.89it/s]


,QuestionID,SubmittedAnswer,StudentID,Score,HomeWork,Question,Database,AnswerKey,schema,full_score,few_mp,num,4o_answer_few
2439,14,USE music; SELECT * FROM album WHERE artist...,dsa285,1.0,Assignment 2C: Filtering Queries,Using the id of Rolling Stones from Question C...,music,SELECT * FROM album WHERE artist_id = 4,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,7528,None
125,0,SELECT artist_name FROM artist ORDER BY arti...,cas760,1.0,Assignment 2B: Selection Queries,"List all the names of the artists, without the...",music,SELECT artist_name FROM artist ORDER BY artist...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,7477,None
8427,52,"SELECT\tT.carrier, \t\tCOUNT(DISTINCT T.origi...",ac10819,10.0,Final Exam,Flights: Using the table flights.m_ticket_pric...,flights,"SELECT carrier, COUNT(*) AS routes, COUNT(DIST...","[{'airports': ['airport', 'state', 'state_name...",10.0,\n a. We are working with the flights dat...,11276,None
6200,30,"SELECT a.artist_id, a.artist_name, COUNT(a2.al...",ml8055,1.0,Assignment 4: Aggregations,Show the name of the artist and the number of ...,music,"SELECT R.artist_id, R.artist_name, COUNT(*) AS...","[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,13294,None
12567,9,select * from track where artist_id = 3 AND ...,li2116,1.0,Assignment 2C: Filtering Queries,"Find the tracks for the artist with id 3, from...",music,SELECT * FROM track WHERE artist_id =3 and alb...,"[{'album': ['artist_id', 'album_id', 'album_na...",1.0,\n a. We are working with the music datab...,9833,None


In [135]:
# Extract grade and feedback as new columns
data1['4o_grade_few'] = data1['4o_answer_few'].apply(extract_grade)
data1['4o_feedback_few'] = data1['4o_answer_few'].apply(extract_feedback)
data1.head()

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
data1.to_csv('few_completions_final.csv')